# Projeto de Análise de Desigualdade em Saúde no modelo HEAT/OMS
## Bloco 2025/1 = análise do sistema de mortalidade do SUS com agregação por unidades federativas




#### Aspectos gerais e participantes

Projeto Desenvolvido em parceria da MS/SECTICS/DESID com a Organização Mundial de Saúde

Elaborado em linguagem Python, utilizando a biblioteca pysus para acesso ao SIM (Sistema de Informações de Mortalidade do SUS)

Coordenador-geral:

Coordenadores: 

Cientista de dados: Marcos Mesquita

Outros participantes:

Data: 06/12/2024


#### Objetivos

O principal objetivo desse projeto é realizar a avaliação de desigualdade na saúde no Brasil com ênfase para a dimensão de unidades subnacionais, estados e municípios brasileiros, a partir da aplicação do modelo do HEAT, tendo como objetivos específicos:

- Apresentar indicadores de saúde no Brasil desagregados por subunidades nacionais em uma visualização interativa a partir do modelo do HEAT Plus;
- Oferecer um repositório comum e atualizado com dados de desigualdade da saúde no Brasil desagregados por estados e municípios em um modelo sob reconhecimento internacional;
- Permitir exploração e acesso dos dados e da metodologia para outros tipos de análises.

# Etapa 1 - Definição do Bloco

O projeto ocorrerá em blocos sucessivos, nos quais serão definidos o problema e os indicadores a serem analisados.

### Definição do problema do bloco



 H0 - as unidades federativa são variáveis determinantes para mortalidade no país

### Escopo do Bloco 1


* indicador: mortalidade (número de mortes por população, dados de morte do SIM/SUS e dados populacionais pelo Censos de 2010 e 2022)
* localização: todo o Brasil, com agregação de mortalidade por Estados e Municípios
* período: 2010 e 2022 
* doenças: “doenças evitáveis” conforme definido pelo OCDE
* outros critérios de filtro: até 74 anos de idade (em razão da aplicação do critério de "doenças evitáveis")

# Etapa 2 - Análise primária dos dados por bloco

## Aspectos teóricos e técnicos das fontes


#### Aspectos técnicos - ambiente, linguagem e ferramentas principais

A análise será feita utilizando a linguagem Python e a biblioteca Pysus que permite acesso às principais bases do SUS de forma direta.
O Pysus exige um ambiente linux para execução de alguns comandos o que foi possível, mesmo em uma máquina Windows, a partir do WSL do Windows.
Foi criado um ambiente em WSL e o conjunto da extração, tratamento e análise dos dados foi realizado no formato Notebook em um editor VSCode, dentro de um Dev Container: Default Linux Universal.


#### Morte e mortalidade no HEAT

Para entendermos como o HEAT trata a mortalidade, fizemos uma busca de indicadores de mortalidade no WHO Health Inequality Data Repository.

O repositório traz dados e metadados de todos os indicadores utilizados pelo HEAT.
Apresenta um total de 3915 indicadores, distribuídos entre 14 seções.

Foi feita uma revisão geral dos indicadores relacionados à mortalidade, que depois foi refinada pela busca das seguintes palavras : “death”, “mortality”, “fatal”, ‘fatality” ,“Probability of dying”, “suicide”. E foram encontrados 388 indicadores que trazem uma destas expressões na sua descrição.


#### SIM (Sistema de Informações de Mortalidade do SUS)

A mortalidade e os indicadores de morte serão buscados no SIM (Sistema de Informações de Mortalidade do SUS).
O SIM disponibiliza dados de mortes no Brasil desde 1996 por ocorrências, gerido pelo Departamento de Análise de Situação de Saúde, da Secretaria de Vigilância em Saúde do Ministério da Saúde, em conjunto com as Secretarias Estaduais e Municipais de Saúde. Estas últimas responsáveis efetivamente pela coleta dos dados.

O SIM traz informações do indivíduo, como sexo, idade, localidade e causas da morte, classificadas de acordo com o CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde).


#### CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde)

Com o intuito de padronizar a nomenclatura de patologias e com abrangência para todo o mundo, foi elaborado pela Organização Mundial da Saúde (OMS) uma  classificação internacional de doenças, a CID (Classificação Internacional de Doenças e Problemas Relacionadas à Saúde). 

A Classificação é atualizada periodicamente. A versão mais atual é a CID11, publicada em janeiro de 2022.

No entanto, por se tratar de revisão que traz impactos nos registros e que exige considerável atualização das ferramentas de captura das informações, o SIM utiliza versões anteriores da CID.
- CID9 - dados até 1995
- CID10 - a partir de 1996 até a data atual

Em razão do recorte temporal dos dados, de 1998 a 2022, iremos utilizar somente a CID10.
A CID10 é organizada em 22 capítulos e possui, além da classificação de doenças, descrição de códigos que permitem identificar sinais, sintomas, queixas, causas externas e circunstâncias sociais.


#### CID de doenças evitáveis

Como elemento do escopo deste trabalho iremos tratar de doenças evitáveis.

O Brasil, a partir da Secretaria de Vigilância em Saúde apresenta uma classificação de doenças evitáveis. 

Lista de causas evitáveis pode ser encontrada nos links: http://tabnet.datasus.gov.br/cgi/sim/Obitos_Evitaveis_0_a_4_anos.pdf




#### População (Censo)


Os dados de população são os do Censo, acessados pelo site do IBGE, https://sidra.ibge.gov.br/Tabela/9606






### Instalação e importação de bibliotecas

#### Instalação de bibliotecas

In [79]:
# # Costuma ser exigida a instalação destas bibliotecas no primeiro acesso ao Pysus
# !apt-get update  
# !apt-get install libffi-dev  # Install libffi-dev
# !pip install --upgrade pip  # Ensure pip is up to date
# !pip install pysus # Exa
# !pip install gdown
!pip install pylance

#### Importação de bibliotecas

In [4]:
# importar biblioteca básica para manipulação de dados
import pandas as pd
import numpy
import os
import sidrapy
import requests # para carga de endereços web - api
import gdown



In [82]:
# importar bibliotecas para carga dos dados
import pysus
import pylance

from pysus import SIM
from pysus.online_data.SIM import download
from pysus.preprocessing.decoders import translate_variables_SIM
from pysus.preprocessing.SIM import group_and_count, redistribute_missing, redistribute_cid_chapter
from pysus.online_data.SIM import get_CID9_table, get_CID10_table, get_municipios, get_ocupations
from ftplib import FTP
from pathlib import Path  

#### Diretórios - configuração

In [83]:
# Definir diretório padrão para pysus
pysus.online_data.SIM.CACHEPATH="./downloads_sim_original"
# Definir pasta para download de arquivos csv e parquet
import os 

# Define diretório para armazenar arquivos do sim (carregados por pysus) quando o SIM.CACHEPATH não funcionar
os.makedirs('downloads_sim_original', exist_ok=True)  
downloads_sim = "downloads_sim_original"

# Define diretório para armazenar outros arquivos
os.makedirs('downloads_outros', exist_ok=True)  

# Define diretório para armazenar arquivos trabalhados
os.makedirs('files_clean', exist_ok=True)  

#### Acesso ao SIM

In [84]:
sim = SIM().load() # Loads the files from DATASUS

In [87]:
sim.paths

(/dissemin/publicos/SIM/CID10/DORES, /dissemin/publicos/SIM/CID9/DORES)

## Coleta e estruturação dos dados

### Carga de dados SIM

#### Carga SIM principal

In [99]:
# Carrega os dados do SIM
# Definição de variáveis
grupo = "CID10" # Agrupamento por CID10
uf = "BR" # A sigla BR carrega dados de todos as unidades federativas
ano = [2010] # Anos do escopo, 2010 e 2022
local = downloads_sim # Local para armazenar os arquivos, definido nas etapas anteriores

dobr = download(grupo,uf,ano,local) # Carrega os dados do SIM
#dobr2010 = download("cid10","br",2022,downloads_sim)
dobr2010 

DOBR2022.parquet: 100%|██████████| 3.78M/3.78M [01:22<00:00, 45.6kB/s]


downloads_sim_original/DOBR2022.parquet

#### Carga SIM auxiliar

O SIM traz algumas tabelas auxiliares de CID10 e municípios. Trazemos estas informações para algum tipo de batimento, mas para estas informações iremos utilizar fontes externas.

In [10]:
# Download de tabela auxiliar de CID10
cid10 = get_CID10_table()
cid10.head()

2024-12-26 13:54:33.967 | DEBUG    | pysus.online_data.SIM:get_CID10_table:116 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2024-12-26 13:54:33.997 | DEBUG    | pysus.online_data.SIM:get_CID10_table:120 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2024-12-26 13:54:34.602 | INFO     | pysus.online_data.SIM:get_CID10_table:146 - Data stored as parquet at ./downloads_sim_original/SIM_CID10_.parquet
2024-12-26 13:54:34.603 | DEBUG    | pysus.online_data.SIM:get_CID10_table:149 - CID10.DBF removed


ModuleNotFoundError: No module named 'numpy.rec'

ModuleNotFoundError: No module named 'numpy.rec'

In [11]:
# Download de tabela auxiliar de CID10
munic = get_municipios()
print(munic.head())

2024-12-26 13:55:15.591 | DEBUG    | pysus.online_data.SIM:get_municipios:208 - Stablishing connection with ftp.datasus.gov.br.
220 Microsoft FTP Service
2024-12-26 13:55:15.623 | DEBUG    | pysus.online_data.SIM:get_municipios:212 - Changing FTP work dir to: /dissemin/publicos/SIM/CID10/TABELAS
2024-12-26 13:55:16.535 | INFO     | pysus.online_data.SIM:get_municipios:238 - Data stored as parquet at ./downloads_sim_original/SIM_CADMUN_.parquet
2024-12-26 13:55:16.536 | DEBUG    | pysus.online_data.SIM:get_municipios:241 - CADMUN.DBF removed


ModuleNotFoundError: No module named 'numpy.rec'

### Carga externa ao SIM

Iremos carregar informações de fontes externas.



##### CIDs doenças evitáveis - SVS

In [14]:
# Download de tabela tratada de CIDs evitáveis de acordo com documentos da SVS

url = 'https://docs.google.com/spreadsheets/d/1YWurDqnBDMz4ACOEctY9uuQkEm7uSRAogzZETQrLCXk/edit?usp=drive_link'
# Convert the Google Sheets URL to a CSV export URL
csv_url = url.replace('/edit?usp=drive_link', '/export?format=csv')

# Read the CSV data into a pandas DataFrame
df = pd.read_csv(csv_url)

# Now you can work with the DataFrame 'df'
df.head()


,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


In [6]:
# Salvar arquivo csv
df.to_csv('downloads_outros/cid10_evit_svs.csv', index=False)

In [8]:
cid_evit_ocde = pd.read_csv('downloads_outros/cid10_evit_svs.csv')
cid_evit_ocde.info()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   CID10_EVIT           322 non-null    object
 1   CID10_EVIT_CLASS     321 non-null    object
 2   cid10_evit_nome      321 non-null    object
 3   cid10_evit_atributo  321 non-null    object
 4   cid10_evit_cod_pai   321 non-null    object
dtypes: object(5)
memory usage: 15.4+ KB


##### CIDs doenças evitáveis - OCDE

In [16]:
# Download de tabela tratada de CIDs evitáveis de acordo com documentos da OCDE
url = 'https://docs.google.com/spreadsheets/d/1X8AH0zs4PNc_bUpZ3tBvFw2HGpgb9CjShOkwXQ0so_I/edit?usp=sharing'
# Convert the Google Sheets URL to a CSV export URL
csv_url = url.replace('/edit?usp=sharing', '/export?format=csv')

# Read the CSV data into a pandas DataFrame
ocde_evit = pd.read_csv(csv_url)

# Now you can work with the DataFrame 'df'
ocde_evit.head()

,avoid_flag,prevent_flag,treat_flag,Group,Causes of deaths,Rationale for inclusion,Range,cid_pai
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [18]:
# Salvar arquivo csv
ocde_evit.to_csv('downloads_outros/cid10_evit_ocde.csv', index=False)

##### População Censo 

In [26]:
# Os dados populacionais são os dos Censos de 2010 e 2022
# Coletamos variáveis de sexo, raça e faixa etária, o que não excedeu os limites do uso do API do SIDRA, então iremos carregar o download feito a posteriori

# ID do arquivo no Google Drive
file_id = '1ohplLORDk_TaAQNQwW7TUjZ10OLxiA2f'

# Baixa o arquivo para o ambiente de execução do Colab
output_file = 'downloaded_file.csv' 
gdown.download(id=file_id, output=output_file, quiet=False)

# Lê o arquivo CSV usando o pandas, pulando as 4 primeiras linhas
censo_pop = pd.read_csv(output_file, skiprows=3)

Downloading...
From: https://drive.google.com/uc?id=1ohplLORDk_TaAQNQwW7TUjZ10OLxiA2f
To: /workspaces/heat_br_bloco1/downloaded_file.csv
100%|██████████| 11.0M/11.0M [00:00<00:00, 19.0MB/s]
/tmp/ipykernel_3561/3438096824.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_pop = pd.read_csv(output_file, skiprows=3)


In [27]:
censo_pop.head()


,Ano,Nível,Cód.,Município,Cor ou raça,Sexo,Total,0 a 4 anos,75 a 79 anos,80 a 84 anos,85 a 89 anos,90 a 94 anos,95 a 99 anos,100 anos ou mais
0,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Branca,Homens,5311,430,63,33,11,2,-,-
1,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Branca,Mulheres,5038,446,62,26,14,2,2,-
2,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Preta,Homens,693,23,9,7,2,1,1,-
3,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Preta,Mulheres,466,21,4,3,1,-,-,-
4,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Amarela,Homens,96,4,2,-,-,-,-,-


In [28]:
# Salvar arquivo de população do Censo
censo_pop.to_csv('downloads_outros/censo_pop.csv')

## Tratamento de Dados

#### Tratamento de População Censo

In [49]:
# Carga de arquivo de População do Censo
censo_pop = pd.read_csv('downloads_outros/censo_pop.csv')

/tmp/ipykernel_3561/3371380752.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  censo_pop = pd.read_csv('downloads_outros/censo_pop.csv')


In [50]:
censo_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111420 entries, 0 to 111419
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        111420 non-null  int64  
 1   Ano               111419 non-null  object 
 2   Nível             111407 non-null  object 
 3   Cód.              111400 non-null  float64
 4   Município         111400 non-null  object 
 5   Cor ou raça       111400 non-null  object 
 6   Sexo              111400 non-null  object 
 7   Total             111400 non-null  object 
 8   0 a 4 anos        111400 non-null  object 
 9   75 a 79 anos      111400 non-null  object 
 10  80 a 84 anos      111400 non-null  object 
 11  85 a 89 anos      111400 non-null  object 
 12  90 a 94 anos      111400 non-null  object 
 13  95 a 99 anos      111400 non-null  object 
 14  100 anos ou mais  111400 non-null  object 
dtypes: float64(1), int64(1), object(13)
memory usage: 12.8+ MB


In [46]:
censo_pop.columns

Index(['Unnamed: 0', 'Ano', 'Nível', 'Cód.', 'Município', 'Cor ou raça',
       'Sexo', 'Total', '0 a 4 anos', '75 a 79 anos', '80 a 84 anos',
       '85 a 89 anos', '90 a 94 anos', '95 a 99 anos', '100 anos ou mais'],
      dtype='object')

In [52]:
# Colunas para transformar em inteiro
cols_age = ['Total','0 a 4 anos', '75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']
# para todas as colunas identificar substituir '-' por 0
censo_pop_clean01 = censo_pop.copy()
censo_pop_clean01[cols_age] = censo_pop_clean01[cols_age].replace('-',0)

# transformar todas as colunas definidas para formato integer
censo_pop_clean01[cols_age] = censo_pop_clean01[cols_age].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
censo_pop_clean01.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111420 entries, 0 to 111419
Data columns (total 15 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        111420 non-null  int64  
 1   Ano               111419 non-null  object 
 2   Nível             111407 non-null  object 
 3   Cód.              111400 non-null  float64
 4   Município         111400 non-null  object 
 5   Cor ou raça       111400 non-null  object 
 6   Sexo              111400 non-null  object 
 7   Total             111420 non-null  int64  
 8   0 a 4 anos        111420 non-null  int64  
 9   75 a 79 anos      111420 non-null  int64  
 10  80 a 84 anos      111420 non-null  int64  
 11  85 a 89 anos      111420 non-null  int64  
 12  90 a 94 anos      111420 non-null  int64  
 13  95 a 99 anos      111420 non-null  int64  
 14  100 anos ou mais  111420 non-null  int64  
dtypes: float64(1), int64(9), object(5)
memory usage: 12.8+ MB


In [63]:
# Gerar coluna de população total até 74 anos
cols_age_over75 = ['75 a 79 anos', '80 a 84 anos', '85 a 89 anos','90 a 94 anos', '95 a 99 anos', '100 anos ou mais']

censo_pop_clean02 = censo_pop_clean01.copy()

# Subtrair a população de 75 anos ou mais da população total
censo_pop_clean02['pop_under74'] = censo_pop_clean02['Total'] - censo_pop_clean02[cols_age_over75].sum(axis=1)

# Excluir colunar cols_age_over75 e 0 a 4 anos
censo_pop_clean02.drop(columns=cols_age_over75, inplace=True)

censo_pop_clean02.head()


,Unnamed: 0,Ano,Nível,Cód.,Município,Cor ou raça,Sexo,Total,0 a 4 anos,pop_under74
0,0,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Branca,Homens,5311,430,5202
1,1,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Branca,Mulheres,5038,446,4932
2,2,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Preta,Homens,693,23,673
3,3,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Preta,Mulheres,466,21,458
4,4,2010,MU,1100015.0,Alta Floresta D'Oeste (RO),Amarela,Homens,96,4,94


In [55]:
# Gerar informação de UF a partir do split de 'Município', com o dividor '('
censo_pop_clean03 = censo_pop_clean02.copy()
censo_pop_clean03['UF'] = censo_pop_clean03['Município'].str.split('(', expand=True)[1].str.replace(')','')
censo_pop_clean03['Município'] = censo_pop_clean03['Município'].str.split('(', expand=True)[0]
censo_pop_clean03.head()


,Unnamed: 0,Ano,Nível,Cód.,Município,Cor ou raça,Sexo,Total,0 a 4 anos,pop_under74,UF
0,0,2010,MU,1100015.0,Alta Floresta D'Oeste,Branca,Homens,5311,430,5202,RO
1,1,2010,MU,1100015.0,Alta Floresta D'Oeste,Branca,Mulheres,5038,446,4932,RO
2,2,2010,MU,1100015.0,Alta Floresta D'Oeste,Preta,Homens,693,23,673,RO
3,3,2010,MU,1100015.0,Alta Floresta D'Oeste,Preta,Mulheres,466,21,458,RO
4,4,2010,MU,1100015.0,Alta Floresta D'Oeste,Amarela,Homens,96,4,94,RO


In [65]:
# Excluir colunas Unnamed, Nivel
censo_pop_clean04 = censo_pop_clean03.copy()

# Alterar nome da coluna Município para mun_nome, Cód para mun_cod
censo_pop_clean04.rename(columns={'Município':'mun_nome','Cód.':'mun_cod','Total':'pop_total','0 a 4 anos':'pop_under5','Cor ou raça':'cor'}, inplace=True)

# Excluir colunas Unnamed, Nivel
censo_pop_clean04.drop(columns=['Unnamed: 0','Nível'], inplace=True)
censo_pop_clean04.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111420 entries, 0 to 111419
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Ano          111419 non-null  object 
 1   mun_cod      111400 non-null  float64
 2   mun_nome     111400 non-null  object 
 3   cor          111400 non-null  object 
 4   Sexo         111400 non-null  object 
 5   pop_total    111420 non-null  int64  
 6   pop_under5   111420 non-null  int64  
 7   pop_under74  111420 non-null  int64  
 8   UF           111400 non-null  object 
dtypes: float64(1), int64(3), object(5)
memory usage: 7.7+ MB


In [67]:
# Excluir caracteres da coluna mun_cod com o delimitador '.'
censo_pop_clean04['mun_cod'] = censo_pop_clean04['mun_cod'].str.replace('.','')

# Mudar formato de coluna num_cod para string
censo_pop_clean04['mun_cod'] = censo_pop_clean04['mun_cod'].astype(str)

censo_pop_clean04.head()


,Ano,mun_cod,mun_nome,cor,Sexo,pop_total,pop_under5,pop_under74,UF
0,2010,11000150,Alta Floresta D'Oeste,Branca,Homens,5311,430,5202,RO
1,2010,11000150,Alta Floresta D'Oeste,Branca,Mulheres,5038,446,4932,RO
2,2010,11000150,Alta Floresta D'Oeste,Preta,Homens,693,23,673,RO
3,2010,11000150,Alta Floresta D'Oeste,Preta,Mulheres,466,21,458,RO
4,2010,11000150,Alta Floresta D'Oeste,Amarela,Homens,96,4,94,RO


##### Arquivo pronto


In [68]:
# Arquivo de População Censo Tratado (pronto para merge)
censo_pop_clean = censo_pop_clean04.to_csv('files_clean/censo_pop_clean.csv', index=False)

#### Tratamento do arquivo principal

O dicionário do SIM está em "./Estrutura_SIM_para_CD.pdf"

Trabalhamos o CID presente no campo CAUSABAS.

Em razão do escopo da População Estimada (que apresenta dados por Município, mas não de outras variáveis de desigualdade) a primeira versão será um recorte da base DOBR com os dados de CID ('CAUSABAS'), idade (a partir do tratamento de 'DTNASC'), e de Município (a partir do tratamento de 'CODMUNRES').



In [100]:
# Função para adicionar o campo de ano
def add_year_column(file_path):
    year = file_path[-12:-8]
    df = pd.read_parquet(file_path)
    df['ANO'] = year
    return df

# Lista de arquivos parquet
arquivos_parquet = ['downloads_sim_original/DOBR2022.parquet']

# Ler, adicionar o campo de ano e concatenar os arquivos parquet
dobr_df = pd.concat([add_year_column(arquivo) for arquivo in arquivos_parquet])

# Resetar o índice
dobr_df.reset_index(drop=True, inplace=True)

# Exibir as primeiras linhas do DataFrame atualizado
dobr_df.head()

,ORIGEM,TIPOBITO,DTOBITO,HORAOBITO,NATURAL,CODMUNNATU,DTNASC,IDADE,SEXO,RACACOR,...,TPRESGINFO,TPNIVELINV,NUDIASINF,DTCADINF,MORTEPARTO,DTCONCASO,FONTESINF,ALTCAUSA,CONTADOR,ANO
0,1,2,13052022,2000,815,150330,22102000,421,1,4,...,,,,,,,,,390001,2022
1,1,2,13052022,2004,,,02071997,424,1,1,...,,,,,,,,,390002,2022
2,1,2,13052022,1830,815,150330,15032001,421,1,4,...,,,,,,,,,390003,2022
3,1,2,13052022,0300,815,150020,16121998,423,1,2,...,,,,,,,,,390004,2022
4,1,2,13052022,0200,815,150020,21101992,429,1,4,...,,,,,,,,,390005,2022


##### Reduzir o arquivo principal apenas para campos referenciados para População Estimada

In [101]:
# Filtro de campos a serem utilizados - versão 1
dobr_cid_mun_idade = dobr_df[['CAUSABAS','DTNASC', 'DTOBITO','CODMUNRES']]
dobr_cid_mun_idade.head()


,CAUSABAS,DTNASC,DTOBITO,CODMUNRES
0,V629,22102000,13052022,150330
1,X990,02071997,13052022,150010
2,X939,15032001,13052022,150330
3,X930,16121998,13052022,150020
4,X959,21101992,13052022,150020


##### Tratamento de datas

In [102]:
# Função para transformar datas e excluir registros inválidos
def transform_date_columns(df, date_columns):
    invalid_rows = []
    
    for date_column in date_columns:
        for index, row in df.iterrows():
            try:
                df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')
            except ValueError:
                invalid_rows.append(index)
    
    # Excluir as linhas identificadas em invalid_rows
    df = df.drop(invalid_rows)
    
    # Apresentar a conta das linhas com erros
    print(f"Linhas com erro: {len(invalid_rows)}")
    
    return df

# Transformar os campos DTNASC e DTOBITO
dobr_cid_mun_idade = transform_date_columns(dobr_cid_mun_idade, ['DTNASC', 'DTOBITO'])

# Verificar a nova base
print(dobr_cid_mun_idade.info())
dobr_cid_mun_idade.head()

/tmp/ipykernel_3561/3653329788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')
/tmp/ipykernel_3561/3653329788.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, f'{date_column}_trat'] = pd.to_datetime(row[date_column], format='%d%m%Y').strftime('%d/%m/%Y')


Linhas com erro: 3343
<class 'pandas.core.frame.DataFrame'>
Index: 1540923 entries, 0 to 1544265
Data columns (total 6 columns):
 #   Column        Non-Null Count    Dtype 
---  ------        --------------    ----- 
 0   CAUSABAS      1540923 non-null  object
 1   DTNASC        1540923 non-null  object
 2   DTOBITO       1540923 non-null  object
 3   CODMUNRES     1540923 non-null  object
 4   DTNASC_trat   1540923 non-null  object
 5   DTOBITO_trat  1540923 non-null  object
dtypes: object(6)
memory usage: 82.3+ MB
None


,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat
0,V629,22102000,13052022,150330,22/10/2000,13/05/2022
1,X990,02071997,13052022,150010,02/07/1997,13/05/2022
2,X939,15032001,13052022,150330,15/03/2001,13/05/2022
3,X930,16121998,13052022,150020,16/12/1998,13/05/2022
4,X959,21101992,13052022,150020,21/10/1992,13/05/2022


In [ ]:
# Salvar arquivo após seleção de colunas e transformação de idade
dobr_treat_parcial = dobr_cid_mun_idade.to_parquet('files_clean/dobr_treat_parcial.parquet')

In [104]:
# pegar 20 linhas aleatórias
df_teste = dobr_cid_mun_idade.sample(20)
df_teste

,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat
498802,J189,15082006,16102022,210923,15/08/2006,16/10/2022
1283928,I10,07101953,25122022,311850,07/10/1953,25/12/2022
1273543,I619,03011945,22082022,330170,03/01/1945,22/08/2022
322245,N189,28101945,31082022,320520,28/10/1945,31/08/2022
926185,C719,01061961,10062022,431320,01/06/1961,10/06/2022
206445,W069,11111933,05102022,315460,11/11/1933,05/10/2022
257286,J841,18011954,18082022,350570,18/01/1954,18/08/2022
752929,I619,03031953,28112022,310620,03/03/1953,28/11/2022
492369,I500,07011947,30102022,330350,07/01/1947,30/10/2022
336270,I10,19041996,30102022,310400,19/04/1996,30/10/2022


##### Criação de faixas de idade

In [116]:
dobr_cid_mun_idade = pd.read_csv('files_clean/dobr_cid_mun_idade.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'files_clean/dobr_parcial.csv'

In [106]:
# Calcular a diferença entre DTNASC_trat e DTOBITO_trat como OBITO_IDADE
dobr_cid_mun_idade['OBITO_IDADE'] = (pd.to_datetime(dobr_cid_mun_idade['DTOBITO_trat'], format='%d/%m/%Y') - pd.to_datetime(dobr_cid_mun_idade['DTNASC_trat'], format='%d/%m/%Y')).dt.days // 365


In [107]:
# Gerar uma coluna com faixa etária, considerando a idade do óbito, 'under74' para menores de 75 anos e 'over75' para 75 anos ou mais
dobr_cid_mun_idade['OBITO_FAIXA_ETARIA'] = numpy.where(dobr_cid_mun_idade['OBITO_IDADE'] < 75, 'under74', 'over75')
dobr_cid_mun_idade.head()

,CAUSABAS,DTNASC,DTOBITO,CODMUNRES,DTNASC_trat,DTOBITO_trat,OBITO_IDADE,OBITO_FAIXA_ETARIA
0,V629,22102000,13052022,150330,22/10/2000,13/05/2022,21,under74
1,X990,02071997,13052022,150010,02/07/1997,13/05/2022,24,under74
2,X939,15032001,13052022,150330,15/03/2001,13/05/2022,21,under74
3,X930,16121998,13052022,150020,16/12/1998,13/05/2022,23,under74
4,X959,21101992,13052022,150020,21/10/1992,13/05/2022,29,under74


In [110]:
# GroupBy para Faixas de Idade
dobr_cid_mun_idade.groupby('OBITO_FAIXA_ETARIA').size()


OBITO_FAIXA_ETARIA
over75     666902
under74    874021
dtype: int64

In [111]:
# Excluir quem na OBITO_FAIXA_ETARIA for 'over75'
dobr_clean = dobr_cid_mun_idade[dobr_cid_mun_idade['OBITO_FAIXA_ETARIA'] == 'under74']

# Resetar o índice
dobr_clean.reset_index(drop=True, inplace=True)


In [113]:
# Criar o campo "ANO" com os últimos 4 caracteres de "DTOBITO_trat"
dobr_clean['ANO'] = dobr_clean['DTOBITO_trat'].str[-4:]

# Listar e filtrar colunas a serem utilizadas
dobr_clean = dobr_clean[['CAUSABAS', 'CODMUNRES', 'OBITO_FAIXA_ETARIA', 'ANO']]

# Converter todos os campos para string
dobr_clean = dobr_clean.astype(str)


/tmp/ipykernel_3561/3943671286.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dobr_clean['ANO'] = dobr_clean['DTOBITO_trat'].str[-4:]


##### Arquivo pronto


In [115]:
# Salvar arquivo em csv
dobr_clean.to_csv('files_clean/dobr_clean.csv', index=False)

In [114]:
# Verificar as primeiras linhas do DataFrame atualizado
dobr_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 874021 entries, 0 to 874020
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   CAUSABAS            874021 non-null  object
 1   CODMUNRES           874021 non-null  object
 2   OBITO_FAIXA_ETARIA  874021 non-null  object
 3   ANO                 874021 non-null  object
dtypes: object(4)
memory usage: 26.7+ MB


Resultados:

A partir do critério de qualidade dos dados de DTNASC e DTOBITO, foram excluídos:
* 3.343 registros que não apresentavam o campo compatível com o formato data
* 666.902 registros de pessoas com idade de óbito 80 ou mais


O Dataframe principal tem no total 874.021  registros.

### Merge de dados de CID e População

In [29]:
print(dobr_clean['CAUSABAS'].info())
print(cid_evit_ocde['CAUSABAS'].info())

<class 'pandas.core.series.Series'>
RangeIndex: 2004769 entries, 0 to 2004768
Series name: CAUSABAS
Non-Null Count    Dtype 
--------------    ----- 
2004769 non-null  object
dtypes: object(1)
memory usage: 15.3+ MB


In [34]:
df_a = dobr_clean
column_a = 'CAUSABAS'
df_b = cid_evit_ocde
column_b = 'CID_AVOID'
column_return = 'CID_PAI'



# Função para verificar a correspondência e retornar a classificação
def verificar_correspondencia(row):
    for index, row_b in df_b.iterrows():
        if row[column_a].startswith(row_b[column_b]):
            return row_b[column_return]
    return 'causa não evitável'

# Aplicando a função a cada linha do DataFrame A
df_a['classificacao'] = df_a.apply(verificar_correspondencia, axis=1)

print(df_a)

KeyboardInterrupt: 

In [2]:
# Exibição de informações da tabela de CID evitáveis de acordo com o MS/SVS
cid_evit_svs = pd.read_excel('downloads_outros/SVS_cid_evitavel_clean.xlsx')
cid_evit_svs.head()

,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


In [3]:
# Exibição de informações da tabela de CID evitáveis de acordo com a OCDE
cid_evit_ocde = pd.read_excel('downloads_outros/OCDE_avoidable_mortality.xlsx')
cid_evit_ocde.head()

,Avoidable mortality,Preventable mortality,Treatable mortality,Group,Causes of deaths,Rationale for inclusion,Coluna2,ICD.IO
0,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Intestinal diseases,Most of these infections can be prevented thro...,(A00-A09),A0
1,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A35
2,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A36
3,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,"Diphtheria, Tetanus, Poliomyelitis",Most of these infections can be prevented thro...,"(A35, A36, A80)",A80
4,Avoidable mortality,Preventable mortality,NaN,Infectious diseases,Whooping cough,Most of these infections can be prevented thro...,(A37),A37


In [17]:
popest_ibge_clean.head()


,Unnamed: 0,POPULACAO,MUNCOD,MUNNOME,ANO,UFCOD,UFNOM
0,1,23167,1100015,Alta Floresta D'Oeste - RO,2018,11,Rondônia
1,2,22945,1100015,Alta Floresta D'Oeste - RO,2019,11,Rondônia
2,3,22728,1100015,Alta Floresta D'Oeste - RO,2020,11,Rondônia
3,4,22516,1100015,Alta Floresta D'Oeste - RO,2021,11,Rondônia
4,5,22853,1100015,Alta Floresta D'Oeste - RO,2024,11,Rondônia


In [16]:
dobr_clean.head()

,CAUSABAS,CODMUNRES,IDADE_FAIXA,ANO
0,B342,110020,55-59,2021
1,I64,250750,70-74,2021
2,C785,250750,60-64,2021
3,I64,250320,70-74,2021
4,J189,250320,70-74,2021


In [93]:
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, popest_ibge_clean, left_on=['CODMUNRES', 'ANO'], right_on=['MUNCOD', 'ANO'], how='left')

# Exibir as primeiras linhas do dataframe resultante
print(merged_df.head())

  CAUSABAS CODMUNRES IDADE_FAIXA   ANO  Unnamed: 0 POPULACAO MUNCOD MUNNOME  \
0     B342    110020       55-59  2021         NaN       NaN    NaN     NaN   
1     I64     250750       70-74  2021         NaN       NaN    NaN     NaN   
2     C785    250750       60-64  2021         NaN       NaN    NaN     NaN   
3     I64     250320       70-74  2021         NaN       NaN    NaN     NaN   
4     J189    250320       70-74  2021         NaN       NaN    NaN     NaN   

  UFCOD UFNOM  
0   NaN   NaN  
1   NaN   NaN  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN  


In [ ]:
# Create a new column in dobr_clean to indicate if CAUSABAS starts with any value in cid_evit_ocde.ICD.IO
dobr_clean['is_avoidable'] = dobr_clean['CAUSABAS'].apply(lambda x: any(x.startswith(icd) for icd in cid_evit_ocde['ICD.IO']))

# Display the updated dataframe
dobr_clean.head()

In [ ]:
#pesquisar se dobr_clean.causabas possui os primeiros caracteres de cid_evit_ocde.icd.10
#pesquisar se dobr_clean.causabas possui os primeiros caracteres de cid_evit_ocde.icd.10



In [20]:
#Fazer um merge entre dobr_clean e cid_evit_ocde, pesquisando se a coluna CAUSABAS começa com o valor da coluna ICD.IO
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, cid_evit_ocde, left_on=['CAUSABAS'], right_on=['ICD.IO'], how='left')

In [21]:
merged_df.head()

,CAUSABAS,CODMUNRES,IDADE_FAIXA,ANO,Avoidable mortality,Preventable mortality,Treatable mortality,Group,Causes of deaths,Rationale for inclusion,Coluna2,ICD.IO
0,B342,110020,55-59,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,I64,250750,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,C785,250750,60-64,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,I64,250320,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J189,250320,70-74,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Realizar o merge dos dataframes
merged_df = pd.merge(dobr_clean, cid_evit_ocde, left_on=['CAUSABAS'], right_on=['MUNCOD', 'ANO'], how='left')

# Exibir as primeiras linhas do dataframe resultante
print(merged_df.head())

In [ ]:
# prompt: juntar dados de df e cid10_df
file_left1 = dobr_clean[['CAUSABAS', 'CODMUNRES', 'IDADE_FAIXA','ANO']]
file_right1 = popest_ibge_clean [['UFCOD', 'UFNOM', 'MUNCOD', 'MUNNOME', 'ANO', 'POPULACAO' ]] 
file_right2 = cid_evit_ocde [['Avoidable mortality', 'Preventable mortality', 'Treatable mortality', 
                              'Group', 'Causes of deaths', 'ICD.IO']] 
file_right3 = cid_evit_svs [['CID10_EVIT', 'CID10_EVIT_CLASS', 'cid10_evit_nome','cid10_evit_cod_pai']]

# Merge the dataframes
file_left2 = pd.merge(file_left1, file_right1, left_on='CAUSABAS', right_on='cid10_cod', how='left') # Trazer nome do CID
merged1 = pd.merge(file_left2, file_right2, left_on='CODMUNOCOR', right_on='MUNCOD', how='left')
merged = pd.merge(merged1, file_right2, left_on='CODMUNRES', right_on='MUNCOD', how='left')

# Renomear colunas
merged.rename(columns={'MUNNOME_x':'MUNNOME_OCOR','MUNNOME_y':'MUNNOME_RES'},inplace=True)

# Excluir colunas
merged_clean = merged.drop(columns=['MUNCOD_x','MUNCOD_y','cid10_cod'])
# merged1 = pd.merge(file_left2, file_right2, left_on='CODMUNOCOR', right_on='MUNCOD', how='left').rename(columns={'MUNNOME':'MUNNOME_OCOR'},inplace=True) # Trazer nome de município de ocorrência
# merged = pd.merge(merged1, file_right2, left_on='CODMUNRES', right_on='MUNCOD', how='left').rename(columns={'MUNNOME':'MUNNOME_RES'},inplace=True) # Trazer nome de município de residência
# Display the merged dataframe
# merged_df = pd.DataFrame(merged)
print(merged_clean.info())
merged_clean.head()

In [46]:
# Exibir as colunas dos dataframes do arquivo principal e dos demais arquivos de população estimada e CID evitáveis
cid_evit_svs.head()

,CID10_EVIT,CID10_EVIT_CLASS,cid10_evit_nome,cid10_evit_atributo,cid10_evit_cod_pai
0,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose do sistema nervoso,(A17),A17
1,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tuberculose miliar,(A19),A19
2,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Tétano obstétrico,(A34),A34
3,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Outros tipos de tétano,(A35),A35
4,Causas Evitáveis,Reduzíveis pelas ações de imunoprevenção,Difteria,(A36),A36


### Seleção e tratamento de variáveis da base primária

#### Carregar arquivos concatenados como dataframe

# Etapa 3 - Aplicação de Modelo do HEAT ao Bloco

# Etapa 4 - Reunião de alinhamento - ajustes

# Etapa 5 - Preparação para publicação de resultados